In [9]:
import os

In [2]:
%pwd

'c:\\Users\\KEERTHI RAJ\\Desktop\\KEERTHI RAJ\\PROJECTS\\DLPROJECTS\\Chicken-Disease-Classification-MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\KEERTHI RAJ\\Desktop\\KEERTHI RAJ\\PROJECTS\\DLPROJECTS\\Chicken-Disease-Classification-MLOPS'

### Creating the Entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_directory: Path
    source_URL: str
    local_data_file: Path
    unzip_directory: Path

### Creating the Configurations

In [10]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import readYaml, createDirectories

In [8]:
class ConfiguationManager:

    """
    All these values are taken from the config file
    """
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = readYaml(config_filepath)
        self.params = readYaml(params_filepath)

        createDirectories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
         config = self.config.data_ingestion
         createDirectories([config.root_directory])

         data_ingestion_config = DataIngestionConfig(
             root_directory = config.root_directory,
             source_URL = config.source_URL,
             local_data_file = config.local_data_file,
             unzip_directory = config.unzip_directory
            )
         
         return data_ingestion_config

### Creating the Components

In [11]:
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
from ChickenDiseaseClassifier.utils.common import getSize

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers  = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {getSize(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_directory
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

### Creating the Pipeline

In [15]:
try:
    config = ConfiguationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-31 20:55:58,670: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-05-31 20:55:58,691: INFO:common:yaml file: params.yaml loaded successfully]
[2024-05-31 20:55:58,694: INFO:common:Created Directory at artifacts]
[2024-05-31 20:55:58,695: INFO:common:Created Directory at artifacts/data_ingestion]
[2024-05-31 20:56:00,267: INFO:583930610:artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5BA5:F1970:4302ED:4A9BB3:665A7FAF
Accept-Ranges: bytes
Date: Sat, 01 Jun 2024 01:56:01 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-kdfw8210112-DFW
X-Cache: MISS
X-C